In [1]:
import nltk
#nltk.download('punkt')
import torch
import numpy as np
import datasets
from datasets import load_dataset, load_from_disk
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, HfArgumentParser
#from transformers.integrations import MLflowCallback, AzureMLCallback

```python
# td: FileSystems integration for cloud storage (abfs for Azure Blob service)
#data = load_dataset("csv", data_files={"train": "crypto-news-train.csv", "validation": "crypto-news-validation.csv"})
#data = load_dataset("json", data_files="corpus.json")
cached.save_to_disk(dir_path)
data = load_from_disk(dir_path)
```
az ml data create --file hf-dataset.yml  

In [12]:
# WARNING: long process (xsum, cnn)
#reddit = load_dataset("reddit", cache_dir="../cache")
#cnn = load_dataset("cnn_dailymail", "3.0.0", cache_dir="../cache")
#reddit_tifu = load_dataset("reddit_tifu", "long", cache_dir="../cache")
#samsum = load_dataset("samsum", cache_dir="../cache")
#cnn.save_to_disk("../data/cnn-dm")
#reddit_tifu.save_to_disk("../data/reddit-tifu")
#samsum.save_to_disk("../data/samsum")

DatasetDict({
    train: Dataset({
        features: ['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title'],
        num_rows: 42139
    })
})

In [2]:
xsum = load_from_disk("../data/xsum")
xsum["train"] = xsum["train"].select(range(25600))
xsum["validation"] = xsum["validation"].select(range(5120))
xsum["test"] = xsum["test"].select(range(5120))
xsum

DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 25600
    })
    validation: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 5120
    })
    test: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 5120
    })
})

In [6]:
xsum["train"][888]

{'document': 'Funding concerns had cast doubt over whether the city could remain a host port for the fourth consecutive year.\nAbout 163,000 people were estimated to have enjoyed the Foyle Maritime Festival and Clipper events in 2016.\nThe Â£200,000 deal to see the race come back will be rubber stamped by Derry City and Strabane District Council later this month.\nDespite entering its own yacht in previous years, the council has not yet secured a team for the upcoming race.\nIt said efforts to find private sponsorship would continue but would not be paid by ratepayers.\nClipper Race chairman and founder Sir Robin Knox-Johnston said he was delighted to welcome the city back on to the race route.\n"Over the past six years the city has welcomed our international crew, supporters and partners with open arms and is an exceptional place to visit and do business with," he said.\n"The spirit of Derry-Londonderry and its people is contagious, which has consistently positioned itself as one of t

In [3]:
xsum.save_to_disk("../data/xsum-25600")

In [ ]:
# reddit
# features: ['author', 'subreddit', 'summary', 'text']
# num_rows: 3848330
#reddit = reddit["train"].remove_columns(["id", "subreddit_id", "body", "normalizedBody"])
#reddit = reddit.rename_column("content", "text")
#reddit.save_to_disk("../data/reddit")
#reddit = load_from_disk("../data/reddit")
#reddit

In [ ]:
# reddit-dota
# features: ['author', 'summary', 'text']
# num_rows: 22366
#dota = reddit.filter(lambda x: x["subreddit"] == "DotA2")
#dota = dota.remove_columns("subreddit")

# bad text/summary @ dota[9]
#dota = dota.filter(lambda x: len(x["text"]) > len(x["summary"])) # source length > target length
#dota.flatten_indices().save_to_disk("../data/reddit-dota")
dota = load_from_disk("../data/reddit-dota")
dota

In [ ]:
# reddit-dota-proc
# num_rows: 19949
#max_text_length = 50000
min_summary_length = 20
max_ratio = 0.5
min_ratio = 0.005
#dota = dota.filter(lambda x: len(x["text"]) <= max_text_length)
proc = dota.filter(lambda x: len(x["summary"]) >= min_summary_length)
proc = proc.filter(lambda x: len(x["summary"])/len(x["text"]) <= max_ratio)
proc = proc.filter(lambda x: len(x["summary"])/len(x["text"]) >= min_ratio)
# bertscore filter?
proc.flatten_indices().save_to_disk("../data/reddit-dota-proc")
proc

In [ ]:
# reddit-dota-qtr
# num_rows: 14389
max_text_length = 3200
min_summary_length = 30
# 0.25(max: 128/512) & 0.125(max: 64/512)
max_ratio = 0.25
min_ratio = 0.01
dota = load_from_disk("../data/reddit-dota-proc")
dota = dota.filter(lambda x: len(x["text"]) <= max_text_length)
dota = dota.filter(lambda x: len(x["summary"]) >= min_summary_length)
dota = dota.filter(lambda x: len(x["summary"])/len(x["text"]) <= max_ratio)
dota = dota.filter(lambda x: len(x["summary"])/len(x["text"]) >= min_ratio)
dota.flatten_indices().save_to_disk("../data/reddit-dota-qtr")
dota = load_from_disk("../data/reddit-dota-qtr")
dota